# ``EOCubeSceneCollection`` and virtual time series

``EOCubeSceneCollection`` and ``EOCubeSceneCollectionChunk`` are similar to ``EOCube`` and ``EOCubeChunk`` and have basically all the functionality of these classes but some more specific functionality and behavious.

While ``EOCube`` and ``EOCubeChunk`` do not make any assumption about the input data, ``EOCubeSceneCollection`` and ``EOCubeSceneCollectionChunk`` assume that the input is a collection (usally time series) of scenes where each scene consists of the same bands (including a quality assessment layer). 

Lets get some typical dataset to make it more tangible what we are talking about.

But first we load the packages required in this tutorial and the function for loading a sample dataset.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from pathlib import Path
import rasterio
import seaborn as sns

# from eobox.raster import MultiRasterIO
from eobox import sampledata
from eobox.raster import cube
from eobox.raster import gdalutils

from eobox.raster.utils import cleanup_df_values_for_given_dtype
from eobox.raster.utils import dtype_checker_df

print(cube.__file__)
print(sampledata.__file__)

/home/ben/Devel/Packages/eo-box/raster/eobox/raster/cube.py
/home/ben/Devel/Packages/eo-box/sampledata/eobox/sampledata/__init__.py


## Sample dataset

In [2]:
def get_sampledata(year):
    dataset = sampledata.get_dataset("lsts")
    layers_paths = [Path(p) for p in dataset["raster_files"]]
    layers_df = pd.Series([p.stem for p in layers_paths]).str.split("_", expand=True) \
    .rename({0: "sceneid", 1:"band"}, axis=1)

    layers_df["date"] = pd.to_datetime(layers_df.sceneid.str[9:16], format="%Y%j")
    layers_df["uname"] = layers_df.sceneid.str[:3] + "_" + layers_df.date.dt.strftime("%Y-%m-%d") + "_" + layers_df.band.str[::] 
    layers_df["path"] = layers_paths

    layers_df = layers_df.sort_values(["date", "band"])
    layers_df = layers_df.reset_index(drop=True)

    layers_df_year = layers_df[(layers_df.date >= str(year)) & (layers_df.date < str(year+1))]
    layers_df_year = layers_df_year.reset_index(drop=True)
    return layers_df_year

The sample data we are loading here contains 23 scenes each of which consists of three bands (*b3*, *b4*, *b5*) and a QA (quality assessment) band (here *fmask*).
This is a typical starting point for nowadays *using-all-available-pixels* EO analysis tasks. 

In [3]:
df_layers = get_sampledata(2008)
display(df_layers.head())
display(df_layers.tail(1))
df_layers.band.value_counts()

,sceneid,band,date,uname,path
0,LT50350322008110PAC01,b3,2008-04-19,LT5_2008-04-19_b3,/home/ben/Devel/Packages/eo-box/sampledata/eob...
1,LT50350322008110PAC01,b4,2008-04-19,LT5_2008-04-19_b4,/home/ben/Devel/Packages/eo-box/sampledata/eob...
2,LT50350322008110PAC01,b5,2008-04-19,LT5_2008-04-19_b5,/home/ben/Devel/Packages/eo-box/sampledata/eob...
3,LT50350322008110PAC01,fmask,2008-04-19,LT5_2008-04-19_fmask,/home/ben/Devel/Packages/eo-box/sampledata/eob...
4,LE70350322008118EDC00,b3,2008-04-27,LE7_2008-04-27_b3,/home/ben/Devel/Packages/eo-box/sampledata/eob...


,sceneid,band,date,uname,path
91,LE70350322008342EDC00,fmask,2008-12-07,LE7_2008-12-07_fmask,/home/ben/Devel/Packages/eo-box/sampledata/eob...


b3       23
b5       23
fmask    23
b4       23
Name: band, dtype: int64

## ``EOCubeSceneCollection``

### Initialization

Typically we want to derive temporal features from these kind of dataset which involves masking out invalid pixels.
Invalid pixels are typically covered by clouds, cloud shadows, saturated data, pixels which are outside the the sensed area, snow, etc.
Usually, these categories are usually included in the QA layer.

For example, *fmask* is a typical Landsat QA-like (or pre classification) layer which has the following categories:

      0 - clear land
      1 - clear water
      2 - cloud
      3 - snow
      4 - shadow
    255 - NoData

As a result we initialize the ``EOCubeSceneCollection`` as follows:

In [4]:
df_layers=df_layers
chunksize=2**5
variables=["b3", "b4", "b5"]
qa="fmask"
qa_valid=[0, 1]

scoll = cube.EOCubeSceneCollection(df_layers=df_layers, 
                                   chunksize=chunksize, 
                                   variables=variables, 
                                   qa=qa, 
                                   qa_valid=qa_valid 
                                  )
scoll.df_layers.head()

,sceneid,band,date,uname,path
0,LT50350322008110PAC01,b3,2008-04-19,LT5_2008-04-19_b3,/home/ben/Devel/Packages/eo-box/sampledata/eob...
1,LT50350322008110PAC01,b4,2008-04-19,LT5_2008-04-19_b4,/home/ben/Devel/Packages/eo-box/sampledata/eob...
2,LT50350322008110PAC01,b5,2008-04-19,LT5_2008-04-19_b5,/home/ben/Devel/Packages/eo-box/sampledata/eob...
3,LT50350322008110PAC01,fmask,2008-04-19,LT5_2008-04-19_fmask,/home/ben/Devel/Packages/eo-box/sampledata/eob...
4,LE70350322008118EDC00,b3,2008-04-27,LE7_2008-04-27_b3,/home/ben/Devel/Packages/eo-box/sampledata/eob...


### Calculation of temporal features

#### Virtual time series

In [5]:
dst_dir = "./xxx_uncontrolled/ls2008_vts4w"
dst_pattern = "./xxx_uncontrolled/ls2008_vts4w/ls2008_vts4w_{date}_{var}.vrt"

freq = '4W'
idx_virtual = pd.date_range(start='2008-01-01', end="2008-12-31", freq=freq)
idx_virtual

scoll.create_virtual_time_series(idx_virtual=idx_virtual,
                                 dst_pattern=dst_pattern,
                                 dtypes="int16",
                                 compress="lzw",
                                 nodata=None,
                                 num_workers=8)

4it [00:04,  1.23s/it]                       


Create a time series layer stack (VRT) for each variable. 

In [6]:
for var in scoll.variables:
    input_file_list = list(list(Path(dst_dir).glob(f"*{var}*")))
    input_file_list = np.sort(input_file_list)
    output_file = Path(dst_dir) / "time_series_stacks" / f"ls_2008_vts__2008__{var}__vts4w.vrt"
    print(output_file)
    gdalutils.buildvrt(input_file_list, output_file, relative=True, separate=True)
    

xxx_uncontrolled/ls2008_vts4w/time_series_stacks/ls_2008_vts__2008__b3__vts4w.vrt
xxx_uncontrolled/ls2008_vts4w/time_series_stacks/ls_2008_vts__2008__b4__vts4w.vrt
xxx_uncontrolled/ls2008_vts4w/time_series_stacks/ls_2008_vts__2008__b5__vts4w.vrt


#### Statistical features

Such as mean, std, min, max, percentiles, iqr, regression slope, ...

**TODO**

## Custom processes

If you need temporal features that are not already implemented you can use the ``EOCubeSceneCollectionChunk`` class to develop your own function and the ``apply_and_write_by_variable`` method of ``EOCubeSceneCollectionChunk`` to process all chunks with the. 
How to do this on the example of the virtual time series is described in this section.

### Developing with ``EOCubeSceneCollectionChunk``

We can get a ``EOCubeSceneCollectionChunk`` which represents a samller spatial chunk of the data. 

In [7]:
sc_chunk = scoll.get_chunk(0)
sc_chunk

The size of the chunk should be such that we can hold the data in memory and do something with it.

Then we can read the data as with the ``EOCubeChunk``.

In [8]:
sc_chunk = sc_chunk.read_data()
print(f"Read the data as 3D array (# pixels x, # pixels y, # layers): {sc_chunk.data.shape}")

Read the data as 3D array (# pixels x, # pixels y, # layers): (32, 32, 92)


We can convert it in a dataframe as with the ``EOCubeChunk`` (note that the column names are the *unames* of the ``df_layers`` dataframe).

In [9]:
sc_chunk = sc_chunk.convert_data_to_dataframe()
sc_chunk.data.head()

uname,LT5_2008-04-19_b3,LT5_2008-04-19_b4,LT5_2008-04-19_b5,LT5_2008-04-19_fmask,LE7_2008-04-27_b3,LE7_2008-04-27_b4,LE7_2008-04-27_b5,LE7_2008-04-27_fmask,LT5_2008-05-05_b3,LT5_2008-05-05_b4,...,LT5_2008-10-28_b5,LT5_2008-10-28_fmask,LE7_2008-11-21_b3,LE7_2008-11-21_b4,LE7_2008-11-21_b5,LE7_2008-11-21_fmask,LE7_2008-12-07_b3,LE7_2008-12-07_b4,LE7_2008-12-07_b5,LE7_2008-12-07_fmask
0,4467,5026,548,3,3907,4862,584,3,3707,4329,...,1788,0,-9999,-9999,-9999,255,2376,3708,665,3
1,4297,4686,571,3,16000,4683,584,3,3680,4297,...,1650,0,-9999,-9999,-9999,255,2340,3657,711,3
2,4242,4720,548,3,16000,4358,562,3,3574,4133,...,1512,0,-9999,-9999,-9999,255,2233,3251,665,3
3,3419,3765,455,3,2591,3493,518,3,2875,3609,...,890,0,-9999,-9999,-9999,255,1519,2282,529,3
4,2362,2943,409,3,1978,2731,430,3,2011,2657,...,510,0,-9999,-9999,-9999,255,692,1309,347,3


We can also get the data of the bands separated.
Note that here we also get the *QA* data.

In [10]:
sc_chunk = sc_chunk.read_data_by_variable(mask=False) # the default is mask=True, see below!
print(f"Keys of sc_chunk.data dictionary: {sc_chunk.data.keys()}")
for key in sc_chunk.data.keys():
    print(f"___________________________\n{key}\n------")
    display(sc_chunk.data[key].head(3))

Keys of sc_chunk.data dictionary: dict_keys(['b3', 'b4', 'b5', 'fmask'])
___________________________
b3
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,4467,3907,3707,1274,-9999,651,-9999,426,240,313,...,919,2481,340,283,480,250,930,894,-9999,2376
1,4297,16000,3680,1354,-9999,596,-9999,426,222,313,...,799,2387,340,283,460,182,968,767,-9999,2340
2,4242,16000,3574,1861,-9999,568,-9999,426,240,313,...,557,2368,340,302,479,182,892,680,-9999,2233


___________________________
b4
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,5026,4862,4329,2215,-9999,2891,-9999,3128,3571,3460,...,2389,4189,2970,3140,3075,534,2477,2250,-9999,3708
1,4686,4683,4297,2280,-9999,2857,-9999,3062,3498,3294,...,2353,4067,2932,2979,2856,492,2301,1995,-9999,3657
2,4720,4358,4133,2803,-9999,2790,-9999,3030,3278,3160,...,2167,3944,2748,2738,2724,450,2168,1791,-9999,3251


___________________________
b5
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,548,584,572,1961,-9999,1720,-9999,1492,1325,1339,...,1155,2218,1253,1146,1472,160,979,1788,-9999,665
1,571,584,506,1916,-9999,1608,-9999,1448,1369,1294,...,1057,2122,1303,1121,1420,189,949,1650,-9999,711
2,548,562,572,1607,-9999,1675,-9999,1491,1369,1317,...,909,2027,1278,1097,1315,160,739,1512,-9999,665


___________________________
fmask
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,3,3,3,0,255,0,255,0,0,0,...,0,4,0,0,0,2,3,0,255,3
1,3,3,0,0,255,0,255,0,0,0,...,0,4,0,0,0,2,3,0,255,3
2,3,3,0,0,255,0,255,0,0,0,...,0,4,0,0,0,2,0,0,255,3


But usually we want to mask the data. 
This is the default of the method.
Note that here we *do not get* the *QA* data.
Now all the values which are not valid (``qa_valid``) according to the *QA* layer are masked out (*NaN*).  

In [11]:
sc_chunk = sc_chunk.read_data_by_variable()
print(f"Keys of sc_chunk.data dictionary: {sc_chunk.data.keys()}")
for key in sc_chunk.data.keys():
    print(f"___________________________\n{key}\n------")
    display(sc_chunk.data[key].head(3))

Keys of sc_chunk.data dictionary: dict_keys(['b3', 'b4', 'b5'])
___________________________
b3
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,NaN,NaN,NaN,1274,NaN,651.0,NaN,426,240.0,313,...,919.0,NaN,340,283.0,480.0,NaN,NaN,894,NaN,NaN
1,NaN,NaN,3680.0,1354,NaN,596.0,NaN,426,222.0,313,...,799.0,NaN,340,283.0,460.0,NaN,NaN,767,NaN,NaN
2,NaN,NaN,3574.0,1861,NaN,568.0,NaN,426,240.0,313,...,557.0,NaN,340,302.0,479.0,NaN,892.0,680,NaN,NaN


___________________________
b4
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,NaN,NaN,NaN,2215,NaN,2891.0,NaN,3128,3571.0,3460,...,2389.0,NaN,2970,3140.0,3075.0,NaN,NaN,2250,NaN,NaN
1,NaN,NaN,4297.0,2280,NaN,2857.0,NaN,3062,3498.0,3294,...,2353.0,NaN,2932,2979.0,2856.0,NaN,NaN,1995,NaN,NaN
2,NaN,NaN,4133.0,2803,NaN,2790.0,NaN,3030,3278.0,3160,...,2167.0,NaN,2748,2738.0,2724.0,NaN,2168.0,1791,NaN,NaN


___________________________
b5
------


date,2008-04-19 00:00:00,2008-04-27 00:00:00,2008-05-05 00:00:00,2008-05-21 00:00:00,2008-05-29 00:00:00,2008-06-06 00:00:00,2008-06-14 00:00:00,2008-06-22 00:00:00,2008-06-30 00:00:00,2008-07-08 00:00:00,...,2008-08-09 00:00:00,2008-08-17 00:00:00,2008-08-25 00:00:00,2008-09-02 00:00:00,2008-09-18 00:00:00,2008-09-26 00:00:00,2008-10-12 00:00:00,2008-10-28 00:00:00,2008-11-21 00:00:00,2008-12-07 00:00:00
0,NaN,NaN,NaN,1961,NaN,1720.0,NaN,1492,1325.0,1339,...,1155.0,NaN,1253,1146.0,1472.0,NaN,NaN,1788,NaN,NaN
1,NaN,NaN,506.0,1916,NaN,1608.0,NaN,1448,1369.0,1294,...,1057.0,NaN,1303,1121.0,1420.0,NaN,NaN,1650,NaN,NaN
2,NaN,NaN,572.0,1607,NaN,1675.0,NaN,1491,1369.0,1317,...,909.0,NaN,1278,1097.0,1315.0,NaN,739.0,1512,NaN,NaN


Now with this data we can do smart stuff, e.g. building a virtual time series by interpolating the data at pre-defined dates.
We might want to do that to be able to create spatially contiguous (temporal) features without missing data over large areas.
Because with such data we can build models.

Thus, we need the pre-defined dates. 
Let's create the dates of a time series with four week frequency in 2018. 

In [12]:
freq = '4W'
idx_virtual = pd.date_range(start='2008-01-01', end="2008-12-31", freq=freq)
idx_virtual

DatetimeIndex(['2008-01-06', '2008-02-03', '2008-03-02', '2008-03-30',
               '2008-04-27', '2008-05-25', '2008-06-22', '2008-07-20',
               '2008-08-17', '2008-09-14', '2008-10-12', '2008-11-09',
               '2008-12-07'],
              dtype='datetime64[ns]', freq='4W-SUN')

Then we need the function that does the calculation. 
It should 

* take a dataframe such as the ones above and 
* return a dataframe which has the pixels again in the rows but can differ in the columns of course.

In [13]:
def create_virtual_time_series(df_var, idx_virtual, num_workers=1, verbosity=0):

    def _create_virtual_time_series_core(df, idx_virtual, verbosity=0):

        if verbosity:
            print("Shape of input dataframe:", df.shape)

        transpose_before_return = False
        if isinstance(df.columns, pd.DatetimeIndex):
            transpose_before_return = True
            df = df.transpose()

        # define the virtual points in the time series index
        # idx_virtual = df.resample(rule).asfreq().index
        if not df.index.is_unique:
            if verbosity:
                print(f"Aggregating (max) data with > observations per day: {df.index[df.index.duplicated()]}")
            df = df.groupby(level=0).max()
            assert df.index.is_unique

        if verbosity:
            print("Length of virtual time series:" ,len(idx_virtual))
        # add the existing time series points to the virtual time series index 
        idx_virtual_and_data = idx_virtual.append(df.index).unique()
        idx_virtual_and_data = idx_virtual_and_data.sort_values()
        if verbosity:
            print("Length of virtual and data time series:" ,len(idx_virtual_and_data))
        # extend the time series data such that it contains all existing and virtual time series points 
        df = df.reindex(index=idx_virtual_and_data)
        # interpolate between dates and forward/backward fill edges with closest values
        df = df.interpolate(method='time')
        df = df.bfill()
        df = df.ffill()
        df = df.loc[idx_virtual]

        if transpose_before_return:
            df = df.transpose()
        if verbosity:
            print("Shape of output dataframe:", df.shape)
        return df

    if (num_workers > 1) or (num_workers == -1):
        import dask.dataframe as dd
        df_var = dd.from_pandas(df_var, npartitions=num_workers)
        df_result = df_var.map_partitions(_create_virtual_time_series_core, 
                                          idx_virtual=idx_virtual, 
                                          verbosity=verbosity)
        df_result = df_result.compute(scheduler='processes', num_workers=num_workers)
    else:
        df_result = _create_virtual_time_series_core(df_var, 
                                                     idx_virtual, 
                                                     verbosity=verbosity)
    return df_result

Now we can make a test run with the chunk.

In [14]:
results = {}
for var in sc_chunk.variables:
    results[var] = create_virtual_time_series(sc_chunk.data[var], idx_virtual, num_workers=1, verbosity=0)
    results[var].columns = pd.MultiIndex.from_product([[var], results[var].columns])
results = pd.concat([results[var] for var in sc_chunk.variables], axis=1)
results.head(3)

b3                                                         \
  2008-01-06 2008-02-03 2008-03-02 2008-03-30 2008-04-27 2008-05-25   
0     1274.0     1274.0     1274.0     1274.0     1274.0    1118.25   
1     3680.0     3680.0     3680.0     3680.0     3680.0    1164.50   
2     3574.0     3574.0     3574.0     3574.0     3574.0    1537.75   

                                               ...         b5             \
  2008-06-22 2008-07-20 2008-08-17 2008-09-14  ... 2008-03-30 2008-04-27   
0      426.0      275.0      629.5     430.75  ...     1961.0     1961.0   
1      426.0      324.0      569.5     415.75  ...      506.0      506.0   
2      426.0      310.0      448.5     434.75  ...      572.0      572.0   

                                                                     \
  2008-05-25 2008-06-22 2008-07-20 2008-08-17 2008-09-14 2008-10-12   
0    1900.75     1492.0      987.5     1204.0    1390.50     1661.6   
1    1839.00     1448.0     1065.0     1180.0    1345.25     1558.0   
2    1624.00     1491.0     1121.0     1093.5    1260.50      739.0   

                         
  2008-11-09 2008-12-07  
0     1788.0     1788.0  
1     1650.0     1650.0  
2     1512.0     1512.0  

[3 rows x 39 columns]

If everything works as we want, we want to save the data.
Therefore we need destination paths for each result layer (or column in our data representation). 
Note that we need only paths for the full spatial dataset (``EOCubeSceneCollection``) and not the chunk.

**TODO: MORE INFO HERE ON HOW THE PATH NAMES WORK!**

### Processing a custom function with ``EOCubeSceneCollection``

In [15]:
dst_paths = {}
for var in variables:
    dst_paths[var] = []
    for date in idx_virtual:
        dst_paths[var].append(dst_pattern.format(**{"var":var, "date":date.strftime("%Y-%m-%d")}))
print(f"First and last path (total {len(dst_paths)}):\n- {dst_paths[variables[0]][0]}\n- ...\n- {dst_paths[variables[-1]][-1]}")
assert (len(idx_virtual) * len(sc_chunk.data)) == sum([len(dst_paths[var]) for var in variables])

First and last path (total 3):
- ./xxx_uncontrolled/ls2008_vts4w/ls2008_vts4w_2008-01-06_b3.vrt
- ...
- ./xxx_uncontrolled/ls2008_vts4w/ls2008_vts4w_2008-12-07_b5.vrt


In [16]:
scoll.apply_and_write_by_variable(mask=True,
                                  fun=create_virtual_time_series, 
                                  dst_paths=dst_paths,
                                  dtypes="int16",
                                  compress="lzw",
                                  nodata=None,
                                  idx_virtual=idx_virtual,
                                  num_workers=8,
                                  verbosity=0)

  0%|          | 0/3 [00:00<?, ?it/s]

b3: 4 / 4 chunks already processed and skipped.
b4: 4 / 4 chunks already processed and skipped.
b5: 4 / 4 chunks already processed and skipped.
